In [1]:
!pip install -U langchain-ollama langchain-openai

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!ls /content/drive/MyDrive/review_dataset

sample_submission.csv  w_review_train.csv
test_file.csv	       w_review_train.parquet


In [4]:
import duckdb

con = duckdb.connect(database=':memory:', read_only=False)

In [5]:
df = con.execute("SELECT * FROM read_csv_auto('/content/drive/MyDrive/review_dataset/w_review_train.csv', parallel=false, encoding='UTF-8', ignore_errors=true)").fetchdf()

df.head()

,column0,column1
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...,3
1,อาหารที่นี่เป็นอาหารจีนแคะที่หากินยากในบ้านเรา...,4
2,ปอเปี๊ยะสด ทุกวันนี้รู้สึกว่าหากินยาก (ร้านที่...,3
3,รัานคัพเค้กในเมืองไทยมีไม่มาก หลายๆคนอาจจะสงสั...,5
4,อร่อย!!! เดินผ่านDigital gatewayทุกวัน ไม่ยักร...,5


In [6]:
df.shape

(40000, 2)

In [7]:
con.execute("COPY df TO '/content/drive/MyDrive/review_dataset/w_review_train.parquet' (FORMAT PARQUET)")

In [8]:
parquet_avg_rating_direct = con.execute("SELECT AVG(column1) FROM '/content/drive/MyDrive/review_dataset/w_review_train.parquet' WHERE column0 LIKE '%อร่อย%'").fetchone()[0]
parquet_avg_rating_direct

3.8313027179006562

In [9]:
parquet_file_path = '/content/drive/MyDrive/review_dataset/w_review_train.parquet'

In [10]:
parquet_avg_rating_direct = con.execute(f"SELECT AVG(column1) FROM '{parquet_file_path}' WHERE column0 LIKE '%อร่อย%'").fetchone()[0]
parquet_avg_rating_direct

3.8313027179006562

In [11]:
coffee_keywords = [
    'กาแฟ', 'คาเฟ่', 'coffee', 'cafe', 'เอสเปรสโซ่', 'ลาเต้',
    'มอคค่า', 'คาปูชิโน่', 'เฟรนช์เพรส', 'cold brew', 'drip',
    'espresso', 'latte', 'americano', 'macchiato', 'brew',
    'specialty', 'slow bar', 'ร้านกาแฟ', 'ร้านคาเฟ่', 'กาแฟสด',
    'เมนูกาแฟ', 'กาแฟนม', 'กาแฟดำ', 'ร้านบาริสต้า', 'barista', 'coffee shop'
]

where_clause = " OR ".join([f"column0 LIKE '%{keyword}%'" for keyword in coffee_keywords])


In [12]:
%%time
coffee_reviews_df = con.execute(f"SELECT * FROM '{parquet_file_path}' WHERE {where_clause}").fetchdf()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 2.09 s, sys: 37 ms, total: 2.13 s
Wall time: 2.91 s


In [13]:
csv_file_path = '/content/drive/MyDrive/review_dataset/w_review_train.csv'

In [14]:
%%time
coffee_reviews_df_csv = con.execute(f"SELECT * FROM read_csv_auto('{csv_file_path}', parallel=false, encoding='UTF-8', ignore_errors=true) WHERE {where_clause}").fetchdf()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 2.33 s, sys: 71.8 ms, total: 2.41 s
Wall time: 3.61 s


In [15]:
coffee_reviews_df.head()

,column0,column1
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...,3
1,วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอา...,4
2,สารภาพว่าไม่เคยคิดจะไปต่อคิวซื้อมากินเองครับ บ...,3
3,ร้านอาหารญี่ปุ่นร้านนี้ ใจจริงไม่อยากแนะนำเลยค...,5
4,เดือนแรกที่เค้าต่อคิวกัน 2 - 3 ชั่วโมง เราก็ว่...,5


In [16]:
coffee_reviews_df.shape

(4981, 2)

In [17]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [18]:

!nohup ollama serve &

nohup: appending output to 'nohup.out'


In [19]:
!ollama pull gemma3:270m

In [20]:
!ollama list

NAME                                  ID              SIZE      MODIFIED               
gemma3:270m                           e7d36fb2c3b3    291 MB    Less than a second ago    
scb10x/typhoon2.1-gemma3-4b:latest    8cfab9097c9d    2.6 GB    6 minutes ago             


In [21]:
!ollama run gemma3:270m "สวัสดี"

สวัสดีค่ะ 😊



In [22]:
!curl -s http://127.0.0.1:11434/api/tags

{"models":[{"name":"gemma3:270m","model":"gemma3:270m","modified_at":"2025-10-19T14:34:13.639483224Z","size":291554930,"digest":"e7d36fb2c3b3293cfe56d55889867a064b3a2b22e98335f2e6e8a387e081d6be","details":{"parent_model":"","format":"gguf","family":"gemma3","families":["gemma3"],"parameter_size":"268.10M","quantization_level":"Q8_0"}},{"name":"scb10x/typhoon2.1-gemma3-4b:latest","model":"scb10x/typhoon2.1-gemma3-4b:latest","modified_at":"2025-10-19T14:27:24.323357144Z","size":2615662352,"digest":"8cfab9097c9d142fa16c7556826c9a7874f4642ac6d575e2d66a4f4979c48b76","details":{"parent_model":"","format":"gguf","family":"gemma3","families":["gemma3"],"parameter_size":"3.9B","quantization_level":"Q4_K_M"}}]}

In [23]:
from langchain_ollama import OllamaLLM
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [24]:

llm = OllamaLLM(model="gemma3:270m")

In [25]:

response = llm.invoke("สวัสดี")
print(response)

สวัสดีค่ะ ยินดีที่ได้รู้จักค่ะ 😊



In [26]:
template = """
คุณเป็นครูสอน Python
คำถาม : {question}
"""

In [27]:
prompt = PromptTemplate(
    input_variables=["question"],
    template=template
)

In [28]:
formatted_prompt = prompt.format(question="เขียนโค้ดหาผลรวมของตัวเลข 1 ถึง 10")
print(formatted_prompt)


คุณเป็นครูสอน Python
คำถาม : เขียนโค้ดหาผลรวมของตัวเลข 1 ถึง 10



In [29]:
chain = prompt | llm

In [30]:
response = chain.invoke({"question": "เขียนโค้ดหาผลรวมของตัวเลข 1 ถึง 10"})

In [31]:
response

'```python\ndef find_sum(numbers):\n  """\n  หาผลรวมของตัวเลขในชุดที่กำหนด\n  """\n  if not numbers:\n    return 0  # กรณีที่ไม่มีตัวเลข\n  return sum(numbers)\n\n# ตัวอย่างการใช้งาน\nnumbers = [1, 2, 3, 4, 5]\nresult = find_sum(numbers)\nprint("ผลรวม:", result)\n```\n\n**คำอธิบายโค้ด:**\n\n1.  **`def find_sum(numbers):`**:  กำหนดชื่อ funzione `find_sum` ซึ่งจะทำการหาผลรวมของตัวเลขในชุด `numbers`\n2.  **`if not numbers:`**:  ตรวจสอบว่ามีตัวเลขในชุด `numbers`หรือไม่\n    *   ถ้าไม่มีตัวเลข จะคืนค่า 0 (กรณีที่ไม่มีตัวเลข)\n    *   ถ้ามีตัวเลขในชุด `numbers` จะ return `sum(numbers)` ซึ่งเป็นค่าคงที่ (ค่าคงที่) ที่เป็นผลรวมของตัวเลขทั้งหมด\n3.  **`return sum(numbers)`**:  ใช้ `sum()` เพื่อหาค่าคงที่ที่รวมตัวเลขทั้งหมดในชุด `numbers`\n4.  **`numbers = [1, 2, 3, 4, 5]`**:  สร้างชุดที่กำหนดเป็น `numbers`\n5.  **`result = find_sum(numbers)`**:  เรียกค่า `find_sum` เพื่อทำการหาผลรวม\n6.  **`print("ผลรวม:", result)`**:  แสดงผลลัพธ์ที่ได้จากโค้ด\n\n**ข้อดีของโค้ดนี้:**\n\n*   **ง่ายและเข้าใจ:**  

In [32]:
from IPython.display import display, Markdown

In [33]:
display(Markdown(response))

```python
def find_sum(numbers):
  """
  หาผลรวมของตัวเลขในชุดที่กำหนด
  """
  if not numbers:
    return 0  # กรณีที่ไม่มีตัวเลข
  return sum(numbers)

# ตัวอย่างการใช้งาน
numbers = [1, 2, 3, 4, 5]
result = find_sum(numbers)
print("ผลรวม:", result)
```

**คำอธิบายโค้ด:**

1.  **`def find_sum(numbers):`**:  กำหนดชื่อ funzione `find_sum` ซึ่งจะทำการหาผลรวมของตัวเลขในชุด `numbers`
2.  **`if not numbers:`**:  ตรวจสอบว่ามีตัวเลขในชุด `numbers`หรือไม่
    *   ถ้าไม่มีตัวเลข จะคืนค่า 0 (กรณีที่ไม่มีตัวเลข)
    *   ถ้ามีตัวเลขในชุด `numbers` จะ return `sum(numbers)` ซึ่งเป็นค่าคงที่ (ค่าคงที่) ที่เป็นผลรวมของตัวเลขทั้งหมด
3.  **`return sum(numbers)`**:  ใช้ `sum()` เพื่อหาค่าคงที่ที่รวมตัวเลขทั้งหมดในชุด `numbers`
4.  **`numbers = [1, 2, 3, 4, 5]`**:  สร้างชุดที่กำหนดเป็น `numbers`
5.  **`result = find_sum(numbers)`**:  เรียกค่า `find_sum` เพื่อทำการหาผลรวม
6.  **`print("ผลรวม:", result)`**:  แสดงผลลัพธ์ที่ได้จากโค้ด

**ข้อดีของโค้ดนี้:**

*   **ง่ายและเข้าใจ:**  โค้ดนี้มีโค้งรับเปลี่ยนได้ง่าย ทำให้ง่ายต่อการเข้าใจและนำไปใช้
*   **การทำงานแบบเรียลไทม์:**  สามารถทำงานได้โดยการเรียกค่า `find_sum`  เมื่อมีตัวเลขในชุด `numbers`
*   **ประสิทธิภาพ:**  ใช้เวลาในการเรียกค่า `sum()`  น้อยกว่าการใช้ `len()` ในกรณีที่ไม่มีตัวเลข
*   **การจัดการกับกรณีที่ไม่มีตัวเลข:**  สามารถจัดการกับกรณีที่ไม่มีตัวเลขได้ (เช่น  `numbers = [1, 2, 3, 4, 5]` ซึ่งจะคืนค่า 0)

**ตัวอย่างการใช้งาน:**

```python
numbers = [1, 2, 3, 4, 5]
result = find_sum(numbers)
print("ผลรวม:", result)
```

Output:

```
ผลรวม: 15
```

**การปรับปรุงเพิ่มเติม:**

*   **การจัดการกับตัวแปร:**  สามารถเพิ่มการจัดการกับตัวแปรเพื่อจัดการกับกรณีที่ตัวเลขไม่รวมกัน (เช่น `numbers = [1, 2, 3, 4, 5]` ซึ่งจะคืนค่า 0)
*   **การจัดการกับตัวเลขที่รวมกัน:**  สามารถจัดการกับตัวเลขที่รวมกันได้เช่นกัน (เช่น `numbers = [1, 2, 3, 4, 5]` ซึ่งจะคืนค่า 10)
*   **การจัดการกับตัวเลขที่ไม่รวมกัน:**  สามารถจัดการกับตัวเลขที่ไม่รวมกันได้เช่นกัน (เช่น `numbers = [1, 2, 3, 4, 5]` ซึ่งจะคืนค่า 0)
*   **การใช้ `if` และ `else`:**  สามารถใช้ `if` เพื่อตรวจสอบว่ามีตัวเลขในชุด `numbers` หรือ `else` เพื่อใช้ `return` ถ้าไม่มีตัวเลข
*   **การแสดงผล:**  สามารถแสดงผลลัพธ์ของ `find_sum`  ได้เช่นกัน (เช่น  `print("ผลรวม:", result)`)



In [34]:
prompt1 = PromptTemplate(
    template="แปลโจทย์เกี่ยวกับ {question}"
)

chain1 = prompt1 | llm

In [35]:
prompt2 = PromptTemplate(
    template="ทำความเข้าใจโจทย์ที่แปลแล้วต่อไปนี้ {translate}"
)

chain2 = prompt2 | llm

In [36]:
prompt3 = PromptTemplate(
    template="แก้โจทย์ตามที่เข้าใจต่อไปนี้ {understand}"
)

chain3 = prompt3 | llm

In [37]:
prompt4 = PromptTemplate(
    template="อธิบายวิธิีทำจากผลลัทธ์ต่อไปนี้ {solve}"
)

chain4 = prompt4 | llm

In [38]:
parser_chain = StrOutputParser()

In [39]:
full_chain = chain1 | chain2 | chain3 | chain4 | parser_chain

In [40]:
result = full_chain.invoke({"question":"2x + 5 = 15"})

In [41]:
result

'เข้าใจแล้วครับ! ผมจะพยายามทำความเข้าใจและแก้โจทย์ตามที่คุณให้มาอย่างละเอียดครับ ผมจะพยายามทำตามขั้นตอนที่คุณให้มา และจะให้ความเข้าใจที่ถูกต้องที่สุดเท่าที่ผมมีเท่าที่สามารถทำได้'

In [42]:
import os
from langchain_openai import ChatOpenAI

In [43]:
result = full_chain.invoke({"question":"2x + 5 = 15"})
result

'วิธิีทำจากผลลัทธ์ต่อไปนี้คือ:\n\n**2x + 5 = 15**\n\n**คำตอบคือ 2x + 5 = 15**'

In [44]:
keyword_prompt = PromptTemplate(
template = """
จาก Review ต่อไปนี้
Extract 3 keywords ที่เป็นตัวแทนของ Review นี้

Review: {review}

ขั้นตอนการ Extract keywords
1. อ่านรีวิวและเข้าใจความหมาย
2. ระบุหัวข้อหลักที่กล่าวถึง
3. เลือก 3 keywords ที่สำคัญที่สุด
4. ตอบในรูปแบบ JSON: {{"keywords": "คำ1, คำ2, คำ3"}}

ให้ทำทีละขั้นตอน
""")

In [45]:
sample_reviews = coffee_reviews_df['column0'].head(2)
sample_reviews

,column0
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...
1,วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอา...


In [46]:
sample_reviews.to_csv('/content/sample_reviews.csv', index=False)

In [47]:
keyword_extract = keyword_prompt | llm | StrOutputParser()

In [48]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    cleaned = (text).replace("**", "").strip()
    results.append({"review": review, "text": cleaned})

CPU times: user 52.6 ms, sys: 4.06 ms, total: 56.6 ms
Wall time: 5.93 s


In [49]:
for result in results:
    print(f"Review {result['review'][:100]}...")
    print(f"text {result['text']}\n")

Review ร้านอาหารใหญ่มากกกกกกก 
เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี 
ห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ...
text ```json
{"keywords": ["ร้านอาหารใหญ่", "เลี้ยวเข้ามาเจอห้องน้ำ", "ห้องทานหลักๆ", "กาแฟ น้ำผึ้ง", "ราคาแพง", "อย่าสั่ง", "ลาบไข่ต้ม", "ไข่มันคาว", "ไม่ประทับใจ", "ทอดมันหัวปลีกรอบ", "อร่อย", "เบิ้ล", "ขนมไทย", "ขนมหวาน", "ฟรี", "ขนมไทย", "แพง", "เกินไป", "รับไม่ไหว", "ว", "กินแล้วมีขนมหวานให้กินฟรีเล็กน้อย"]}
```

Review วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอารีย์

ก็ได้ยินมาบ่อยๆละ จากหลายที่ แต่ที่บ่อยสุดก็จาก ...
text ```json
{
  "keywords": [
    "ลาเต้",
    "ร้อน",
    "แก้วนึง",
    "ร้อน",
    "เสริฟ",
    "โต๊ะ",
    "ริมกระจก",
    "เล่นลายฟองนม",
    "เว้ยยย",
    "ฉีกน้ำตาล",
    "ถ่ายรูป",
    "55+",
    "ซากอารยธรรมลาเต้",
    "เมพพพพ",
    "รสชาติ",
    "กลิ่น",
    "มวยรุ่นเดียวกะ Starbuck"
  ]
}
```



In [50]:
!ollama pull scb10x/typhoon2.1-gemma3-4b

In [51]:
!ollama list

NAME                                  ID              SIZE      MODIFIED               
scb10x/typhoon2.1-gemma3-4b:latest    8cfab9097c9d    2.6 GB    Less than a second ago    
gemma3:270m                           e7d36fb2c3b3    291 MB    29 seconds ago            


In [52]:
llm3 = OllamaLLM(model="scb10x/typhoon2.1-gemma3-4b:latest")

In [53]:
keyword_extract = keyword_prompt | llm3 | StrOutputParser()

In [54]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    cleaned = (text).replace("**", "").strip()
    results.append({"review": review, "text": cleaned})

CPU times: user 104 ms, sys: 12.9 ms, total: 117 ms
Wall time: 16.1 s


In [55]:
for result in results:
    print(f"Review {result['review'][:100]}...")
    print(f"text {result['text']}\n")

Review ร้านอาหารใหญ่มากกกกกกก 
เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี 
ห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ...
text แน่นอนครับ ผมจะทำตามขั้นตอนที่คุณระบุมาเพื่อดึงคำสำคัญจากรีวิวนี้ และตอบในรูปแบบ JSON ดังนี้ครับ

ขั้นตอนที่ 1: อ่านรีวิวและเข้าใจความหมาย

รีวิวนี้พูดถึงประสบการณ์การทานอาหารที่ร้านอาหารแห่งหนึ่ง โดยมีประเด็นหลักๆ ดังนี้:

*   ร้านใหญ่มาก
*   ห้องน้ำอยู่ก่อนถึงห้องอาหาร
*   อาหารบางเมนูราคาแพงเกินไป (เช่น น้ำผึ้ง, พะแนงห่อไข่)
*   อาหารบางเมนูอร่อย (เช่น ทอดมันหัวปลี)
*   มีขนมหวานให้ฟรี

ขั้นตอนที่ 2: ระบุหัวข้อหลักที่กล่าวถึง

หัวข้อหลักที่กล่าวถึง ได้แก่:

*   ขนาดร้าน
*   ราคาอาหาร
*   รสชาติอาหาร (ดี/ไม่ดี)
*   ของหวาน

ขั้นตอนที่ 3: เลือก 3 keywords ที่สำคัญที่สุด

จากหัวข้อหลักที่กล่าวถึง ผมเลือกคำเหล่านี้เป็น keywords ที่สำคัญที่สุด:

1.  ราคา
2.  รสชาติ
3.  ขนาด

ขั้นตอนที่ 4: ตอบในรูปแบบ JSON

```json
{
  "keywords": ["ราคา", "รสชาติ", "ขนาด"]
}
```

หวังว่าคำตอบนี้จะเป็นประโยชน์นะครับ หากมีคำถามเพิ่มเติม หรือต้องการให้ผมช่วยในเรื่องอื่นๆ บอกได้เลยครับ

Review 

In [56]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser

class ReviewKeywords(BaseModel):
    keywords: str = Field(
        ...,  # Required field
        description="3 keywords separated by comma",
        examples=["กาแฟ, บรรยากาศ, ราคา"]
    )

class KeywordsExtraction(BaseModel):
    extracted_qualities: ReviewKeywords

In [57]:
keywords_parser = JsonOutputParser(pydantic_object=KeywordsExtraction)

In [58]:
keyword_extract = keyword_prompt | llm3 | keywords_parser

In [59]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    results.append({"review": review, "text": text})

CPU times: user 181 ms, sys: 11.1 ms, total: 192 ms
Wall time: 16 s


In [60]:
results[0]['text']['keywords']

['ราคา', 'รสชาติ', 'ขนาด']

In [61]:
results[1]['text']['keywords']

['กาแฟ', 'บรรยากาศ', 'วาวี']

In [62]:
llm3 = OllamaLLM(
    model="scb10x/typhoon2.1-gemma3-4b",
    temperature=0.05
)

keyword_extract = keyword_prompt | llm3 | keywords_parser

In [63]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    results.append({"review": review, "text": text})

CPU times: user 165 ms, sys: 7.31 ms, total: 172 ms
Wall time: 14.1 s


In [64]:
results[0]['text']['keywords'], results[1]['text']['keywords']

(['ราคา', 'อาหาร', 'ขนาดร้าน'], ['กาแฟ', 'บรรยากาศ', 'รสชาติ'])

In [65]:
keyword_prompt = PromptTemplate(
    template="""จาก Review ต่อไปนี้
    Extract keywords 3 keywords เท่านั้น ที่เป็นตัวแทนของ Review นี้ได้ดีที่สุด

    Review: {review}

    ตอบในรูปแบบ JSON โดยมีคีย์เป็น 'keywords'
    และค่าเป็นข้อความที่ดึงได้คั่นด้วย comma
    เช่น {{"keywords": "กาแฟ, บรรยากาศ, ราคา"}}

    หากไม่พบข้อมูลให้ตอบ 'ไม่ระบุ'
    """
)

In [66]:
keyword_extract = keyword_prompt | llm3 | keywords_parser

In [67]:
inputs = [{"review": r} for r in sample_reviews]

In [68]:
%%time
results = keyword_extract.batch(inputs)

CPU times: user 24.1 ms, sys: 1.43 ms, total: 25.6 ms
Wall time: 2.48 s


In [69]:
results

[{'keywords': 'ราคา, บรรยากาศ, ไข่ต้ม'},
 {'keywords': 'กาแฟ, บรรยากาศ, รสชาติ'}]

In [70]:
sample_reviews_1000 = coffee_reviews_df['column0'].head(1000).tolist()
inputs_1000 = [{"review": r} for r in sample_reviews_1000]

BATCH_SIZE = 8

results_1000 = []

In [71]:
from tqdm import tqdm

In [72]:
for i in tqdm(range(0, len(inputs_1000), BATCH_SIZE), desc="Extracting keywords"):
    batch = inputs_1000[i:i+BATCH_SIZE]
    outputs = keyword_extract.batch(batch)
    for inp, out in zip(batch, outputs):
        results_1000.append({"review": inp["review"], "text": out})

Extracting keywords: 100%|██████████| 125/125 [20:23<00:00,  9.78s/it]


In [73]:
for result in results_1000[0:10]:
    print(result['text']['keywords'])

ราคา, บรรยากาศ, ไข่ต้ม
กาแฟ, บรรยากาศ, รสชาติ
โดนัท, รสชาติ, ความหวาน
วาซาบิ, ปลาซาบะ, ข้าวหน้าปลา
โดนัท, Original Glazed, คิว
วังพญาไท, นรสิงห์, คาเฟ่
อาหารทะเล, ราคา, บรรยากาศ
กาแฟ, เค้ก, พนักงาน
ฮันนี่ โทสท์, ราคา, รสชาติ
คาเฟ่, วานิลลา, คาโบนาร่า


In [74]:
context_prompt = PromptTemplate(
    template="""จาก Review ต่อไปนี้
    Extract context ใน Review ที่เป็นตัวแทนที่บรรยายคำว่า {keyword} ได้ดีที่สุด

    Review: {review}

    ตอบผลลัพธ์ในรูปแบบ JSON โดยมีคีย์เป็น 'context'
    และค่าเป็นข้อความที่ดึงได้จาก Review

    ตัวอย่าง
    keyword คือ "กาแฟ"
    context: "กาแฟที่นี่อร่อยมาก หอมกลิ่นกาแฟคั่วสด"

    หากไม่พบข้อมูลให้ตอบ 'ไม่ระบุ'
    """
)

In [75]:
sentiment_prompt = PromptTemplate(
    template="""จากข้อความต่อไปนี้ ให้วิเคราะห์ว่าเป็น sentiment แบบไหน
    เลือกจากตัวเลือกเหล่านี้เท่านั้น: "positive", "negative", "neutral"

    ข้อความ: {message}

    ตอบในรูปแบบ JSON โดยมีคีย์เป็น 'sentiment'
    และค่าเป็นหนึ่งใน list ต่อไปนี้ [positive, negative, neutral]

    คำแนะนำ
    - positive คือ ข้อความที่แสดงความพอใจ ชอบ ดี
    - negative คือ ข้อความที่แสดงความไม่พอใจ ไม่ชอบ แย่
    - neutral คือ ข้อความที่ไม่แสดงความรู้สึกชัดเจน เป็นกลาง

    # ตัวอย่าง
    # ข้อความ "อร่อยมาก" → {{"sentiment": "positive"}}
    # ข้อความ "แพงไป" → {{"sentiment": "negative"}}
    # ข้อความ "ปกติ" → {{"sentiment": "neutral"}}
    """
)

In [76]:
class ContextKeyword(BaseModel):
    context: str = Field(..., description="context that best represent this keyword")

class ContexExtraction(BaseModel):
    extracted: ContextKeyword

context_parser = JsonOutputParser(pydantic_object=ContexExtraction)

In [77]:
context_extract = context_prompt | llm3 | context_parser

In [78]:
class Sentiment(BaseModel):
    sentiment: str = Field(..., description="sentiment of review")

class SentimentExtraction(BaseModel):
    extracted: Sentiment


sentiment_parser = JsonOutputParser(pydantic_object=SentimentExtraction)

In [79]:
sentiment_extract = sentiment_prompt | llm3 | sentiment_parser

In [81]:
results_1000[0]['text']['keywords'].split(',')

['ราคา', ' บรรยากาศ', ' ไข่ต้ม']

In [82]:
context_inputs = []

for review_data in tqdm(results_1000, desc="Preparing context inputs"):
    review = review_data['review']
    keywords = review_data['text']['keywords'].split(',')
    for keyword in keywords:
        context_inputs.append({"review": review, "keyword": keyword.strip()})

Preparing context inputs: 100%|██████████| 1000/1000 [00:00<00:00, 295998.87it/s]


In [83]:
len(context_inputs)

3000

In [84]:
context_results = []
for i in tqdm(range(len(results_1000)), desc="Extracting contexts"):
   result = context_extract.batch(context_inputs[i*3:(i*3)+3])
   contexts_list = [d['context'] for d in result]
   results_1000[i]['contexts'] = contexts_list

Extracting contexts: 100%|██████████| 1000/1000 [1:14:09<00:00,  4.45s/it]


In [85]:
results_1000[:2]

[{'review': 'ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี \nห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ้ง ซึ่งก็แค่เอาน้ำผึ้งมาราด แพงเวอร์ อย่าสั่งเลย \nลาบไข่ต้ม ไข่มันคาวอะ เลยไม่ประทับใจเท่าไหร่\nทอดมันหัวปลีกรอบอร่อยต้องเบิ้ล \nพะแนงห่อไข่อร่อยดี เห้ยแต่ราคา 150บาทมันเกินไปนะ รับไม่ไหวว\nเลิกกินแล้วมีขนมหวานให้กินฟรีเล็กน้อย )ขนมไทย) \n\nคงไม่ไปซ้ำ แพงเกิน ',
  'text': {'keywords': 'ราคา, บรรยากาศ, ไข่ต้ม'},
  'contexts': ['ราคา 150 บาทมันเกินไปนะ รับไม่ไหวว',
   'บรรยากาศร้านใหญ่มากกกกกกก',
   'ไข่ต้ม']},
 {'review': 'วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอารีย์\n\nก็ได้ยินมาบ่อยๆละ จากหลายที่ แต่ที่บ่อยสุดก็จาก Twitter ว่ากาแฟที่นี่อร่อยมากกกกกกก\nเรียกว่าระดับแฟนๆ Starbuck ยังต้องเหลียวมามอง ก็ดองมานานจนถึงวันนี้โอกาสเหมาะ ไป\npresent งานที่ตึก Software Park เสร็จก็เลยมาหาไรกินแถวนี้พอดี\nจัดไป.....\n\n\nบรรยากาศรอบๆร้านก็แต่งได้น่านั่ง อารมณ์ประมาณว่าอยู่ในสวนบ้านเราเอง ต้นไม้เยอะดี\nอย่างน้อยก็เป็นสีเขียวเล็กๆ ในใจกลางเมืองได้ละ\nเขียวชอุ่ม\n\nว่าแล้วไม่รอช้

In [86]:
sentiment_inputs = []

for review_data in tqdm(results_1000, desc="Preparing sentiment inputs"):
    contexts = review_data['contexts']
    for context in contexts:
        sentiment_inputs.append({"message": context})

Preparing sentiment inputs: 100%|██████████| 1000/1000 [00:00<00:00, 386928.41it/s]


In [87]:
len(sentiment_inputs)

3000

In [88]:
sentiment_inputs[:6]

[{'message': 'ราคา 150 บาทมันเกินไปนะ รับไม่ไหวว'},
 {'message': 'บรรยากาศร้านใหญ่มากกกกกกก'},
 {'message': 'ไข่ต้ม'},
 {'message': 'กาแฟที่นี่อร่อยมากกกกกกก เรียกว่าระดับแฟนๆ Starbuck ยังต้องเหลียวมามอง'},
 {'message': 'บรรยากาศรอบๆร้านก็แต่งได้น่านั่ง อารมณ์ประมาณว่าอยู่ในสวนบ้านเราเอง ต้นไม้เยอะดี อย่างน้อยก็เป็นสีเขียวเล็กๆ ในใจกลางเมืองได้ละ เขียวชอุ่ม'},
 {'message': 'รสชาติและกลิ่นมันไม่ธรรมดาเลย หอมๆ ถือว่าเป็นมวยรุ่นเดียวกะ Starbuck ได้'}]

In [89]:
for i in tqdm(range(len(results_1000)), desc="Extracting sentiment"):
   result = sentiment_extract.batch(sentiment_inputs[i*3:(i*3)+3])
   sentiment_list = [d['sentiment'] for d in result]
   results_1000[i]['sentiment'] = sentiment_list

Extracting sentiment: 100%|██████████| 1000/1000 [44:11<00:00,  2.65s/it]


In [90]:
results_1000[0:2]

[{'review': 'ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี \nห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ้ง ซึ่งก็แค่เอาน้ำผึ้งมาราด แพงเวอร์ อย่าสั่งเลย \nลาบไข่ต้ม ไข่มันคาวอะ เลยไม่ประทับใจเท่าไหร่\nทอดมันหัวปลีกรอบอร่อยต้องเบิ้ล \nพะแนงห่อไข่อร่อยดี เห้ยแต่ราคา 150บาทมันเกินไปนะ รับไม่ไหวว\nเลิกกินแล้วมีขนมหวานให้กินฟรีเล็กน้อย )ขนมไทย) \n\nคงไม่ไปซ้ำ แพงเกิน ',
  'text': {'keywords': 'ราคา, บรรยากาศ, ไข่ต้ม'},
  'contexts': ['ราคา 150 บาทมันเกินไปนะ รับไม่ไหวว',
   'บรรยากาศร้านใหญ่มากกกกกกก',
   'ไข่ต้ม'],
  'sentiment': ['negative', 'positive', 'neutral']},
 {'review': 'วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอารีย์\n\nก็ได้ยินมาบ่อยๆละ จากหลายที่ แต่ที่บ่อยสุดก็จาก Twitter ว่ากาแฟที่นี่อร่อยมากกกกกกก\nเรียกว่าระดับแฟนๆ Starbuck ยังต้องเหลียวมามอง ก็ดองมานานจนถึงวันนี้โอกาสเหมาะ ไป\npresent งานที่ตึก Software Park เสร็จก็เลยมาหาไรกินแถวนี้พอดี\nจัดไป.....\n\n\nบรรยากาศรอบๆร้านก็แต่งได้น่านั่ง อารมณ์ประมาณว่าอยู่ในสวนบ้านเราเอง ต้นไม้เยอะดี\nอย่างน้อยก็เป็นสีเขียวเล

In [91]:
import pandas as pd

df = pd.DataFrame([
    {
        'review': r['review'],
        **{f'keyword{i+1}': k.strip() for i, k in enumerate(r['text']['keywords'].split(','))},
        **{f'context{i+1}_keyword': c for i, c in enumerate(r['contexts'])},
        **{f'sentiment{i+1}': s for i, s in enumerate(r['sentiment'])}
    }
    for r in results_1000
])

In [92]:
df.to_csv('results.csv', index=False, encoding='utf-8-sig')